In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [50]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON_adjusted.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [51]:
#week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [6]

In [52]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [53]:
len(link_day_minute_Apr_list)

2880

In [54]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 120)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [55]:
np.size(x, 0), np.size(x, 1)

(24, 120)

In [56]:
x[:,:2]

matrix([[ 1443.99877717,  2078.52980901],
        [ 1099.72721994,   807.29565681],
        [ 2565.31983384,  1551.5802079 ],
        [ 2909.59139107,  2822.81436009],
        [ 4053.3066845 ,  6820.55285579],
        [ 3201.53712535,  4443.05197834],
        [ 1892.44529524,  1784.28420188],
        [ 2399.94329716,  2890.55092714],
        [ 3550.74471349,  3504.82672932],
        [ 2647.36200804,  2429.81406954],
        [ 4238.5917148 ,  5334.84344253],
        [ 4634.47641833,  5303.58937705],
        [ 3568.24723832,  2862.76149737],
        [ 2176.2446117 ,  1571.9636429 ],
        [ 3362.03565615,  3247.63387432],
        [  846.37412819,  1238.44069442],
        [ 1285.88371508,  1142.58082736],
        [ 3185.38434363,  3539.64540709],
        [ 4577.79311777,  5356.55936285],
        [ 4798.06931366,  4999.94202849],
        [ 7077.89202537,  7284.36048548],
        [ 4958.11520094,  5243.91324011],
        [ 3384.58964794,  3560.12350877],
        [ 1485.0890194 ,  1163.058

In [57]:
np.size(A,0), np.size(A,1)

(24, 140)

In [58]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 7e+15]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s


GurobiError: Objective Q not PSD (diagonal adjustment of 5.8e+14 would be required)

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.3473679361259703e-10,
 2807.5279406675036,
 1.4789527570941162e-10,
 6.664381284871544e-07,
 1340.185698734798,
 1.117331191008154e-06,
 4.292180266998288e-08,
 0.0008002217637364528,
 883.4208140992027,
 3190.989735905362,
 6.66438366760504e-07,
 581.764308809899,
 1.1173318479776957e-06,
 1.1237312171175182e-06,
 5901.698371039733,
 6786.292240388291,
 6.664382737235706e-07,
 6.664380364676905e-07,
 1841.6143193380456,
 9939.582535475809,
 1.1170939176753907e-06,
 2668.0948797012643,
 1092.223879663977,
 1249.1429758493898,
 2591.0499451079118,
 1929.568514399647,
 1786.1854149768706,
 3397.0359697039776,
 0.05265260077882594,
 0.016221712204705252,
 4452.8343550655845,
 1299.6803823694868,
 4603.180819429563,
 2580.544048707163,
 1031.5373153905373,
 5.1912698171565144e-05,
 5.1912654500474095e-05,
 6241.137400042277,
 688.7632924300725,
 1729.9460846349782,
 5002.914410416363,
 1140.9183564847178,
 6.650884976128917e-07,
 6.650884885248923e-07,
 6377.683168294559,
 3.31551418306